In [ ]:
pip install gTTS SpeechRecognition playsound PyAudio


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.5 MB/s eta 0:00:00
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=65a0c59f1e360feb68fb932c93d0624d5b15a4a46aaa3475cf79292a6f099083
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound
  Attempting uninstall: click
    Found existing installation: click 8.2.0
    Uninstalling click-8.2.0:
      Successfully uninstalled click-8.2.0


In [ ]:
!pip install pipwin
!pipwin install pyaudio


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.0 MB/s eta 0:00:00
  Created wheel for pipwin: filename=pipwin-0.5.2-py2.py3-none-any.whl size=8770 sha256=235734fbb96182dc9886859f74278cb911e1c15cbd623e40c21f86b9f70beb67
  Stored in directory: /root/.cache/pip/wheels/07/45/2e/51ee464b7c3407327439df34c2fc62f978a1e5084aaf23d201
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=81a0bc21e31f7afd86e7f664d7656c1d996ba5109239859542a36f49b2fdccdf
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=25982 sha256=8840edf227770e7d56d97e6f6b356f70d6c26acee9efea84926d715e60a92088
  Stored in directory: /root/.cache/pip/wheels/a5/9a/30/1003e89ab4555b81840ca46d361bf184f1e6ad880cae3b62a9
Successfully b

# SIMPLE TRANSLATION

In [ ]:
# text_speech_speech_text.py

import os
import speech_recognition as sr
from gtts import gTTS
# playsound will not be used, so it can be removed
# from playsound import playsound
from IPython.display import Audio, display # Import Audio and display

def text_to_speech():
    text = input("Enter text to convert to speech: ")
    language = 'en'

    tts = gTTS(text=text, lang=language, slow=False)
    filename = "tts_output.mp3"
    tts.save(filename)
    print("Playing audio...")
    # Use IPython.display.Audio to play the audio directly in the notebook
    display(Audio(filename=filename, autoplay=True)) # Use display with Audio object
    # Keep the file for playback, no need to remove immediately
    # os.remove(filename)  # Clean up - remove this line

def speech_to_text():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()

    with mic as source:
        print("Speak something...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio)
        print("You said: ", text)
    except sr.UnknownValueError:
        print("Sorry, could not understand your voice.")
    except sr.RequestError:
        print("Could not request results; check your internet connection.")

def main():
    print("Welcome to the Text ↔ Speech Converter")
    print("1. Text to Speech")
    print("2. Speech to Text")
    choice = input("Choose an option (1 or 2): ")

    if choice == '1':
        text_to_speech()
    elif choice == '2':
        speech_to_text()
    else:
        print("Invalid choice.")

if __name__ == "__main__":
    main()

Welcome to the Text ↔ Speech Converter
1. Text to Speech
2. Speech to Text
Choose an option (1 or 2): 1
Enter text to convert to speech: hi
Playing audio...


# **Universal Language Speech ↔ Text Converter Using Transformers (XLS-R + TTS)**

In [ ]:
pip install torchaudio transformers gtts gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.0
    Uninstalling click-8.2.0:
      Successfully uninstalled click-8.2.0


gpt

In [ ]:
!pip install gradio

In [ ]:
!pip install gtts

USING TRANSFORMERS (XLS-R)

CODE NO 1

In [ ]:
pip install gradio transformers torchaudio scipy sentencepiece


In [ ]:
import gradio as gr
import torch
import torchaudio
import tempfile
import numpy as np
from scipy.signal import butter, lfilter
from transformers import (
    Wav2Vec2Processor, Wav2Vec2ForCTC,
    SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan,
    M2M100ForConditionalGeneration, M2M100Tokenizer
)

# ========================
# Load STT Model
# ========================
stt_model_name = "facebook/wav2vec2-large-960h-lv60-self"
stt_processor = Wav2Vec2Processor.from_pretrained(stt_model_name)
stt_model = Wav2Vec2ForCTC.from_pretrained(stt_model_name)

# ========================
# Load TTS Model
# ========================
tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
tts_vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Speaker embedding
speaker_embedding = torch.randn((1, 512)) * 0.05
speaker_embedding[:, 100:200] += 0.1
speaker_embedding[:, 300:400] += 0.1

# ========================
# Load Translation Model
# ========================
translation_model_name = "facebook/m2m100_418M"
translator_tokenizer = M2M100Tokenizer.from_pretrained(translation_model_name)
translator_model = M2M100ForConditionalGeneration.from_pretrained(translation_model_name)

# ========================
# Audio Enhancements
# ========================
def apply_high_pass_filter(audio, sample_rate=16000, cutoff=80):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(4, normal_cutoff, btype='high')
    return lfilter(b, a, audio)

def enhance_audio(audio):
    audio = audio / np.max(np.abs(audio))
    audio = apply_high_pass_filter(audio)
    audio = np.tanh(audio * 1.5)
    return audio / np.max(np.abs(audio))

# ========================
# Translation Function
# ========================
def translate_text(text, src_lang="en", tgt_lang="fr"):
    translator_tokenizer.src_lang = src_lang
    encoded = translator_tokenizer(text, return_tensors="pt")
    generated_tokens = translator_model.generate(**encoded, forced_bos_token_id=translator_tokenizer.get_lang_id(tgt_lang))
    return translator_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

# ========================
# Speech to Text with Translation
# ========================
def speech_to_text_translate(audio, target_lang):
    if audio is None:
        return "Please upload audio", ""
    waveform, sample_rate = torchaudio.load(audio)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)

    inputs = stt_processor(waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = stt_model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcribed = stt_processor.batch_decode(predicted_ids)[0]

    translated = translate_text(transcribed, src_lang="en", tgt_lang=target_lang)
    return transcribed, translated

# ========================
# Text to Speech with Translation
# ========================
def text_to_speech_translate(text, language, speed=1.0, pitch=0.0):
    if not text.strip():
        return None
    translated_text = translate_text(text, src_lang="en", tgt_lang=language)
    inputs = tts_processor(text=translated_text, return_tensors="pt")
    with torch.no_grad():
        speech = tts_model.generate_speech(
            inputs["input_ids"],
            speaker_embedding + pitch * 0.2,
            vocoder=tts_vocoder
        )
    audio = enhance_audio(speech.numpy())
    temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    torchaudio.save(temp_file.name, torch.from_numpy(audio).unsqueeze(0), 16000)
    return temp_file.name

# ========================
# Gradio Interface
# ========================
with gr.Blocks(title="Multilingual Universal Translator") as demo:
    gr.Markdown("## 🌍 Universal Speech Translator (Multilingual)")

    with gr.Tab("🎤 Speech to Translated Text"):
        audio_input = gr.Audio(label="Upload Audio", type="filepath")
        target_lang = gr.Dropdown(label="Translate To", choices=["fr", "de", "es", "it", "ru", "zh", "ja"], value="fr")
        stt_btn = gr.Button("Transcribe & Translate")
        stt_text = gr.Textbox(label="Original Text")
        translated_text = gr.Textbox(label="Translated Text")
        stt_btn.click(fn=speech_to_text_translate, inputs=[audio_input, target_lang], outputs=[stt_text, translated_text])

    with gr.Tab("📝 Text to Speech (Translated)"):
        input_text = gr.Textbox(label="Enter English Text")
        tts_lang = gr.Dropdown(label="Translate & Speak In", choices=["fr", "de", "es", "it", "ru", "zh", "ja"], value="fr")
        speed_slider = gr.Slider(minimum=0.5, maximum=2.0, value=1.0, label="Speed")
        pitch_slider = gr.Slider(minimum=-1.0, maximum=1.0, value=0.0, label="Pitch")
        tts_btn = gr.Button("Generate Translated Speech")
        tts_output = gr.Audio(label="Translated Speech")
        tts_btn.click(fn=text_to_speech_translate, inputs=[input_text, tts_lang, speed_slider, pitch_slider], outputs=tts_output)

if __name__ == "__main__":
    demo.launch()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9e536cb123d0ac9990.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


CODE NO 2

In [ ]:
import gradio as gr
import torch
import torchaudio
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    SpeechT5Processor,
    SpeechT5ForTextToSpeech,
    SpeechT5HifiGan
)
# Ensure datasets is installed and import it
!pip install datasets
from datasets import load_dataset
import tempfile
import os # Import os for path handling

# ========================
# 1. Load Speech-to-Text Model (XLS-R)
# ========================
print("🔄 Loading STT model...")
stt_model_id = "jonatasgrosman/wav2vec2-large-xlsr-53-english"  # ✅ use a valid one with vocab
stt_processor = Wav2Vec2Processor.from_pretrained(stt_model_id)
stt_model = Wav2Vec2ForCTC.from_pretrained(stt_model_id)

# ========================
# 2. Load Text-to-Speech Model (SpeechT5)
# ========================
print("🔄 Loading TTS model...")
tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
tts_vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Load speaker embedding - attempt to handle the loading error
print("🔄 Loading speaker embeddings...")
speaker_embedding = None
try:
    # Attempt to load the dataset
    # Adding cache_dir might help if permissions or path issues are the problem,
    # but the error suggests an issue with the LocalFileSystem handler itself.
    # Let's try a simpler approach first without specific cache settings.
    # If the error persists, consider using a different method to get an embedding.
    embeddings_dataset = load_dataset(
        "Matthijs/cmu-arctic-xvectors",
        split="validation",
        # You could try forcing a redownload, but this might not fix the
        # underlying LocalFileSystem issue if it's systemic.
        # download_mode="force_redownload"
        # Or specify a cache directory if the default one has issues
        # cache_dir="./datasets_cache"
    )
    # Access the first embedding
    speaker_embedding = torch.tensor(embeddings_dataset[0]["xvector"]).unsqueeze(0)
    print("✅ Speaker embedding loaded successfully from dataset.")

except NotImplementedError as e:
    print(f"⚠️ NotImplementedError when loading dataset: {e}")
    print("Trying to find an alternative speaker embedding method.")
    # Fallback method: Use a default or example embedding if available
    # SpeechT5 examples sometimes use a fixed embedding or one derived differently.
    # A common practice is to use a precomputed one if the dataset loading fails.
    # For demonstration, we'll use a dummy tensor with the expected shape (1, 512).
    # In a real application, you would need a valid embedding.
    print("Attempting to create a dummy speaker embedding.")
    try:
        # The shape of the cmu-arctic-xvectors embedding is 1, 512
        # Let's create a zero tensor of that shape as a fallback.
        # NOTE: This dummy embedding will likely result in poor speech quality.
        # A proper fix requires successfully loading a real embedding or finding a better source.
        speaker_embedding = torch.zeros(1, 512)
        print("✅ Dummy speaker embedding created. Note: Speech quality may be poor.")
    except Exception as inner_e:
        print(f"❌ Failed to create dummy speaker embedding: {inner_e}")
        speaker_embedding = None # Ensure embedding is None if even dummy creation fails

except Exception as e:
    print(f"⚠️ An unexpected error occurred while loading dataset: {e}")
    print("Trying to find an alternative speaker embedding method.")
    try:
        # Fallback method (same as above, for any unexpected error)
        speaker_embedding = torch.zeros(1, 512)
        print("✅ Dummy speaker embedding created. Note: Speech quality may be poor.")
    except Exception as inner_e:
        print(f"❌ Failed to create dummy speaker embedding on fallback: {inner_e}")
        speaker_embedding = None

# Check if speaker_embedding is None and raise an error if necessary
if speaker_embedding is None:
    raise RuntimeError("Failed to load or create a speaker embedding. Cannot proceed with TTS.")


# ========================
# Function: Speech to text
# ======================
def speech_to_text(audio_path):
    if audio_path is None:
        return "No audio file provided."
    try:
        waveform, sample_rate = torchaudio.load(audio_path)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        if sample_rate != 16000:
            waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)
        inputs = stt_processor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
        with torch.no_grad():
            logits = stt_model(**inputs).logits
        pred_ids = torch.argmax(logits, dim=-1)
        transcription = stt_processor.decode(pred_ids[0])
        return transcription
    except Exception as e:
        # Print traceback for better debugging
        import traceback
        traceback.print_exc()
        return f"⚠️ Error in Speech-to-Text: {str(e)}"


# ========================
# Function: Text to Speech
# ========================
def text_to_speech(text):
    if speaker_embedding is None:
         return "⚠️ Error: Speaker embedding not loaded. Cannot perform Text-to-Speech."
    try:
        inputs = tts_processor(text=text, return_tensors="pt")
        with torch.no_grad():
            # Ensure inputs["input_ids"] and speaker_embedding are on the same device if using GPU
            # device = tts_model.device
            # speech = tts_model.generate_speech(inputs["input_ids"].to(device), speaker_embedding.to(device), vocoder=tts_vocoder)
            # Simple version assuming CPU or default device:
             speech = tts_model.generate_speech(inputs["input_ids"], speaker_embedding, vocoder=tts_vocoder)

        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as f:
            # Ensure speech tensor is on CPU before saving if it was on GPU
            torchaudio.save(f.name, speech.cpu().unsqueeze(0), 16000)
            audio_path = f.name
        return audio_path
    except Exception as e:
        # Print traceback for better debugging
        import traceback
        traceback.print_exc()
        return f"⚠️ Error in Text-to-Speech: {str(e)}"


# ========================
# Gradio Interface
# ========================
with gr.Blocks() as demo:
    gr.Markdown("## 🌍 Multilingual Speech ↔ Text App using Transformers (XLS-R & SpeechT5)")

    with gr.Tab("🎤 Speech to Text"):
        audio_input = gr.Audio(label="Upload Audio (.wav)", type="filepath")
        stt_button = gr.Button("Convert Speech to Text")
        stt_output = gr.Textbox(label="Transcribed Text")
        stt_button.click(fn=speech_to_text, inputs=audio_input, outputs=stt_output)

    with gr.Tab("📝 Text to Speech"):
        text_input = gr.Textbox(label="Enter Text")
        tts_button = gr.Button("Convert Text to Speech")
        tts_output = gr.Audio(label="Generated Speech")
        # Pass speaker_embedding implicitly or ensure it's accessible
        # It's a global variable in this script, so text_to_speech can access it.
        tts_button.click(fn=text_to_speech, inputs=text_input, outputs=tts_output)

# Launch the interface
try:
    demo.launch()
except Exception as e:
    print(f"Error launching Gradio interface: {str(e)}")
    # If Gradio launch fails, print a message
    print("Gradio interface failed to launch. Check the console output for errors.")

🔄 Loading STT model...


preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

🔄 Loading TTS model...
🔄 Loading speaker embeddings...


Generating validation split: 0 examples [00:00, ? examples/s]

⚠️ NotImplementedError when loading dataset: Loading a dataset cached in a LocalFileSystem is not supported.
Trying to find an alternative speaker embedding method.
Attempting to create a dummy speaker embedding.
✅ Dummy speaker embedding created. Note: Speech quality may be poor.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2948bcc58a232c6cdf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**CODE NO 3**

In [ ]:
import gradio as gr
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import tempfile
import numpy as np
from scipy.signal import lfilter, butter  # ✅ FIXED: added butter import

# ========================
# 1. Load Speech-to-Text Model
# ========================
print("Loading speech recognition model...")
try:
    model_name = "facebook/wav2vec2-large-960h-lv60-self"
    stt_processor = Wav2Vec2Processor.from_pretrained(model_name)
    stt_model = Wav2Vec2ForCTC.from_pretrained(model_name)
    print(f"Successfully loaded {model_name}")
except Exception as e:
    print(f"Error loading primary model: {str(e)}")
    try:
        model_name = "facebook/wav2vec2-base-960h"
        stt_processor = Wav2Vec2Processor.from_pretrained(model_name)
        stt_model = Wav2Vec2ForCTC.from_pretrained(model_name)
        print(f"Successfully loaded fallback model {model_name}")
    except Exception as e:
        raise RuntimeError(f"Failed to load any speech recognition model: {str(e)}")

# ========================
# 2. Load Text-to-Speech Model
# ========================
print("Loading TTS model...")
try:
    tts_processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
    tts_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
    tts_vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
    print("TTS model loaded successfully!")
except Exception as e:
    raise RuntimeError(f"Failed to load TTS models: {str(e)}")

# ========================
# 3. Create Speaker Embedding
# ========================
print("Creating improved speaker embedding...")
try:
    speaker_embedding = torch.randn((1, 512)) * 0.05
    speaker_embedding[:, 100:200] += 0.1
    speaker_embedding[:, 300:400] += 0.1
    print("Created improved speaker embedding")
except Exception as e:
    raise RuntimeError(f"Failed to create speaker embedding: {str(e)}")

# ========================
# 4. Audio Enhancement Functions
# ========================
def apply_high_pass_filter(audio, sample_rate=16000, cutoff=80):
    nyquist = 0.5 * sample_rate
    normal_cutoff = cutoff / nyquist
    b, a = butter(4, normal_cutoff, btype='high', analog=False)
    return lfilter(b, a, audio)

def enhance_audio(audio):
    audio = audio / np.max(np.abs(audio))
    audio = apply_high_pass_filter(audio)
    audio = np.tanh(audio * 1.5)
    audio = audio / np.max(np.abs(audio))
    return audio

# ========================
# 5. Speech to Text Function
# ========================
def speech_to_text(audio):
    if audio is None:
        return "Please upload or record audio"

    try:
        waveform, sample_rate = torchaudio.load(audio)

        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        inputs = stt_processor(
            waveform.squeeze().numpy(),
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )

        with torch.no_grad():
            logits = stt_model(**inputs).logits

        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = stt_processor.batch_decode(predicted_ids)[0]

        return transcription

    except Exception as e:
        return f"Error processing audio: {str(e)}"

# ========================
# 6. Text to Speech Function
# ========================
def text_to_speech(text, language="en", speed=1.0, pitch=0.0):
    if not text.strip():
        return None

    try:
        if language != "en":
            text = f"[{language}]{text}[{language}]"

        inputs = tts_processor(text=text, return_tensors="pt")

        if speed != 1.0:
            seq_len = inputs["input_ids"].shape[1]
            new_len = min(int(seq_len / speed), 500)
            inputs["attention_mask"] = torch.ones((1, new_len), dtype=torch.long)

        with torch.no_grad():
            speech = tts_model.generate_speech(
                inputs["input_ids"],
                speaker_embedding + pitch * 0.2,
                vocoder=tts_vocoder
            )

        speech = speech.numpy()
        speech = enhance_audio(speech)

        temp_file = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        temp_path = temp_file.name
        temp_file.close()

        torchaudio.save(temp_path, torch.from_numpy(speech).unsqueeze(0), 16000)
        return temp_path

    except Exception as e:
        print(f"Error in TTS: {e}")
        return None

# ========================
# 7. Gradio Interface
# ========================
with gr.Blocks(title="Universal Speech Translator") as demo:
    gr.Markdown("""
    # 🌐 Universal Speech Translator
    ### Powered by Facebook's wav2vec2 and Microsoft's SpeechT5
    """)

    with gr.Tab("🎤 Speech to Text"):
        gr.Markdown("Supports English and many other languages")
        audio_input = gr.Audio(label="Input Audio", type="filepath")
        stt_button = gr.Button("Convert to Text", variant="primary")
        stt_output = gr.Textbox(label="Transcription", interactive=True)
        stt_button.click(speech_to_text, inputs=audio_input, outputs=stt_output)

    with gr.Tab("📝 Text to Speech"):
        gr.Markdown("Select target language and voice parameters")
        with gr.Row():
            text_input = gr.Textbox(label="Text", placeholder="Type something...")
            language = gr.Dropdown(
                label="Language",
                choices=["en", "es", "fr", "de", "it", "pt", "ru"],
                value="en"
            )
        with gr.Row():
            speed = gr.Slider(0.5, 2.0, value=1.0, label="Speech Speed")
            pitch = gr.Slider(-1.0, 1.0, value=0.0, label="Pitch Adjustment")
        tts_button = gr.Button("Generate Clear Speech", variant="primary")
        tts_output = gr.Audio(label="Output Speech")
        tts_button.click(
            text_to_speech,
            inputs=[text_input, language, speed, pitch],
            outputs=tts_output
        )

# ========================
# 8. Launch Gradio App
# ========================
try:
    demo.launch(share=True)  # ✅ Enables external access
except Exception as e:
    print(f"Error launching Gradio app: {e}")


Loading speech recognition model...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Successfully loaded facebook/wav2vec2-large-960h-lv60-self
Loading TTS model...
TTS model loaded successfully!
Creating improved speaker embedding...
Created improved speaker embedding
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://791876a803aa866927.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00
